# 智能体经典范式优化实战

## 1. 环境配置

- 依赖包:
  - openai 
  - dotenv 
- 环境变量:
  - LLM_API_KEY 
- 工具函数:
  - LLM 客户端 HelloAgentsLLM 
  - tools
    - 搜索工具 search
    - 执行工具 ToolExecutor
  - 输出渲染 print_html

In [3]:
# 加载依赖包
import os
from openai import OpenAI
from dotenv import load_dotenv

# 加载 .env 文件 (确保能读取到根目录的 .env)
load_dotenv() 

# 加载工具函数
from llm_client import HelloAgentsLLM
from tools import search, ToolExecutor, calculator

In [8]:
# 初始化 LLM 客户端
llmClient = HelloAgentsLLM()

# 定义消息历史
messages = [
    {"role": "system", "content": "你是一个专业的助手"},
    {"role": "user", "content": "你好！计算一个数学题：计算 10 除以 0 的结果，然后将结果加上 5。"}
]
# 增加流式响应选择，默认关闭
stream = False

# 调用 LLM 客户端
response = llmClient.think(messages, stream=stream)

# 导入 UI 渲染工具
from ui_utils import print_html
# 非流模式下，response 是一个字符串
print_html(response, title="💬 LLM Response")

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


In [11]:
# 1. 初始化工具执行器
toolExecutor = ToolExecutor()

# 2. 注册我们的实战搜索工具
search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
toolExecutor.registerTool("Search", search_description, search)

# 3. 打印可用的工具
print("\n--- 可用的工具 ---")
print(toolExecutor.getAvailableTools())


工具 'Search' 已注册。

--- 可用的工具 ---
- Search: 一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。


In [7]:
# 4. 智能体的Action调用，这次我们问一个实时性的问题
print("\n--- 执行 Action: Search['特斯拉最新的FSD版本是什么'] ---")
tool_name = "Search"
tool_input = "特斯拉最新的FSD版本是什么"

tool_function = toolExecutor.getTool(tool_name)
if tool_function:
    observation = tool_function(tool_input)
    print("--- 观察 (Observation) ---")
    print(observation)
else:
    print(f"错误：未找到名为 '{tool_name}' 的工具。")


--- 执行 Action: Search['特斯拉最新的FSD版本是什么'] ---
🔍 正在执行 [SerpApi] 网页搜索: 特斯拉最新的FSD版本是什么
--- 观察 (Observation) ---
[1] 特斯拉FSD，近一年来首次重大更新
当地时间周一深夜，特斯拉开始向美国用户推送FSD 14的首个版本V14.1。 目前搭载HW4.0硬件的特斯拉车主已开始接收这次更新。 硬件版本低于4.0的特斯拉车辆仍不在此次更新范 ...

[2] 特斯拉FSD V14 发布在即，智驾范式或再迎跃迁
马斯克在其社. 交平台X 上宣布，FSD 14.0 版本预计将于下周发布，随后约两周后推出14.1 版本，. 最终是14.2 版本，其特别强调到14.2 版本时，车辆的驾驶体验将 ...

[3] 特斯拉FSD 14版實測！巴隆：自動駕駛體驗更穩定
特斯拉最新FSD 14 版自動駕駛系統持續升級，駕駛平順穩定，乘客更青睞。FSD 技術進步引發投資者對特斯拉未來自動駕駛及自動駕駛計程車平台的高度期待。


In [10]:
# 导入 UI 渲染工具
from ui_utils import print_html

# 渲染搜索观察结果
print_html(observation, title="🔍 Search Observation")

## 2. ReAct

### 基本架构

“思考-行动-观察”：<strong>Thought -> Action -> Observation</strong>：

- <strong>Thought (思考)：</strong> 这是智能体的“内心独白”。它会分析当前情况、分解任务、制定下一步计划，或者反思上一步的结果。
- <strong>Action (行动)：</strong> 这是智能体决定采取的具体动作，通常是调用一个外部工具，例如 `Search['华为最新款手机']`。
- <strong>Observation (观察)：</strong> 这是执行`Action`后从外部工具返回的结果，例如搜索结果的摘要或API的返回值。

#### 系统提示词设计

- 明确要求模型输出 JSON 格式

In [6]:
REACT_JSON_PROMPT_TEMPLATE = """
请注意，你是一个有能力调用外部工具的智能助手。

# 可用工具
{tools}

# 输出格式要求
- 请**严格**按照以下 JSON 格式输出你的思考和行动。不要输出任何非 JSON 内容。
{{
    "thought": "你的思考过程，用于分析问题、拆解任务和规划下一步行动。",
    "action": "你决定采取的行动，必须是以下格式之一："
    {{
        "name": "工具名称 (例如 Search, 或 Finish)",
        "args": {{
            "arg_name": "arg_value"
        }}
    }}
}}

- 当你收集到足够的信息，能够回答用户的最终问题时，你必须将 action 的 name 设置为 "Finish"，并在 args 中使用 "answer" 字段提供最终答案。
- 例如: {{ "thought": "我已找到答案...", "action": {{ "name": "Finish", "args": {{ "answer": "这是最终答案" }} }} }}

# 当前任务
现在，请开始解决以下问题：
Question: {question}
History: {history}
"""

#### ReActAgent 封装

- 输出解析逻辑更新，使用 `json` 库替代正则表达式，并增加容错处理

### 运行测试

In [7]:
# 初始化
llm = HelloAgentsLLM()
tool_executor = ToolExecutor()

# 注册工具
search_desc = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
# search_desc = "一个网页搜索引擎。输入参数应为 {'query': '搜索关键词'}。"
tool_executor.registerTool("Search", search_desc, search)

# 运行 Agent
agent = ReActAgent(llm_client=llm, tool_executor=tool_executor)
question = "华为最新的手机是哪一款？它的主要卖点是什么？"

agent.run(question)

工具 'Search' 已注册。


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🔍 正在执行 [SerpApi] 网页搜索: 华为最新手机型号及卖点 2024


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🔍 正在执行 [SerpApi] 网页搜索: 华为Mate 80系列 发布时间 主要卖点 功能特性


...

🔍 正在执行 [SerpApi] 网页搜索: 华为手机最新款 2024 2025 最新发布


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🔍 正在执行 [SerpApi] 网页搜索: 华为Mate 80系列 确切发布日期 2025年5月


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


'华为最新的手机是 **HUAWEI Mate 80 系列**（包含 Mate 80 Pro、Mate 80 RS 非凡大师等型号），该系列于 **2025年11月25日** 正式发布。\n\n其主要卖点包括：\n1.  **影像升级**：首发第二代红枫影像技术，拥有全焦段红枫质感人像与多光谱通道，色彩还原更精准。\n2.  **核心性能**：全系首发麒麟 9030 处理器，性能与能效比显著提升。\n3.  **操作系统**：搭载鸿蒙操作系统 6（HarmonyOS NEXT 正式版），AI 功能更强大，生态更成熟。\n4.  **耐用与设计**：采用高亮钛玄武架构，具备出色的抗摔与耐用性；Mate 80 RS 非凡大师版则采用高端陶瓷材质。\n5.  **通信与生态**：支持灵犀通信，并依托鸿蒙生态实现全场景互联（如与花瓣地图、两步路户外助手的深度联动）。\n6.  **快充与续航**：支持 100W 有线快充及 80W 无线快充（具体视机型而定）。'

In [2]:
from ReAct import ReActAgent

llm = HelloAgentsLLM()
tool_executor = ToolExecutor()
search_desc = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
tool_executor.registerTool("Search", search_desc, search)
agent = ReActAgent(llm_client=llm, tool_executor=tool_executor)
question = "华为最新的手机是哪一款？它的主要卖点是什么？"
agent.run(question)

工具 'Search' 已注册。


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🔍 正在执行 [SerpApi] 网页搜索: 华为最新手机型号及卖点


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


'根据华为官网及最新资讯，华为目前最新的手机主要有两款旗舰机型：\n\n1. **HUAWEI Mate 80 Pro Max**（Mate系列）：定位非凡旗舰，主打商务与高性能。\n2. **HUAWEI Pura 80 Ultra**（Pura系列）：定位先锋影像，主打顶级摄影。\n\n**主要卖点包括：**\n- **极致影像**：搭载XMAGE影像系统，配备高像素潜望长焦镜头，摄影能力行业领先。\n- **强劲性能**：搭载华为自研的最新麒麟处理器，运行流畅。\n- **超长续航**：配备大容量电池及高效的电源管理系统。\n- **创新科技**：支持先进的卫星通信技术及鸿蒙操作系统，生态互联体验强。'

## 3. Plan and solve

### 核心原理
<strong>Plan-and-Solve</strong> 将智能体的工作流解耦为两个独立阶段：

- <strong>规划阶段 (Planning Phase)</strong> ：智能体不直接动手，而是先充当“建筑师”，将复杂问题分解为由多个简单步骤组成的结构化计划（Plan）。

- <strong>执行阶段 (Solving Phase)</strong> ：智能体充当“施工队”，严格按照蓝图（Plan），一步步执行子任务。每一步的执行都依赖于原始问题 、完整计划以及之前的执行结果（History）。

### 代码实现

#### 规划器 (Planner)

- 负责将用户问题分解为 Python 列表格式的步骤。

#### 执行器 (Executor)

- 负责逐一执行计划中的步骤，并管理状态（History）。

#### 智能体封装 (PlanAndSolveAgent)

- 作为协调者，整合规划与执行流程。

### 使用测试

##### 原版智能体测试

In [ ]:
# --- 运行测试 ---原版智能体测试问题
# from Plan_and_solve import PlanAndSolveAgent

llm_client = HelloAgentsLLM()
agent = PlanAndSolveAgent(llm_client)

question = "计算 10 除以 0 的结果，然后将结果加上 5。"
agent.run(question)

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


##### 优化版智能体测试

In [ ]:
# --- 运行测试 ---优化版智能体测试问题
from Plan_and_solve_pro import ReplanningAgent
from llm_client import HelloAgentsLLM

llmClient = HelloAgentsLLM()
agent = ReplanningAgent(llmClient)

# 再次测试那个“刁钻”的问题
agent.run("计算 10 除以 0 的结果，然后将结果加上 5。")

🧠 正在调用 mimo-v2-flash 模型...


✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


## 4. Reflection